Aller sur https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres/


Télécharger le fichier des ventes 2022 (et / ou les années précédentes si vous le souhaitez)


Placer le fichier dans un dossier data

In [1]:
!head data/valeursfoncieres-2022.txt

Identifiant de document|Reference document|1 Articles CGI|2 Articles CGI|3 Articles CGI|4 Articles CGI|5 Articles CGI|No disposition|Date mutation|Nature mutation|Valeur fonciere|No voie|B/T/Q|Type de voie|Code voie|Voie|Code postal|Commune|Code departement|Code commune|Prefixe de section|Section|No plan|No Volume|1er lot|Surface Carrez du 1er lot|2eme lot|Surface Carrez du 2eme lot|3eme lot|Surface Carrez du 3eme lot|4eme lot|Surface Carrez du 4eme lot|5eme lot|Surface Carrez du 5eme lot|Nombre de lots|Code type local|Type local|Identifiant local|Surface reelle bati|Nombre pieces principales|Nature culture|Nature culture speciale|Surface terrain
|||||||000001|03/01/2022|Vente|55000,00|13||RUE|2280|DE LA LIBERTE|1000|BOURG-EN-BRESSE|01|53||AM|102||7|24,10|||||||||1|2|Appartement||24|1|||
|||||||000001|03/01/2022|Vente|143000,00||||B010|CHAMP COCHET|1480|SAVIGNEUX|01|398||ZE|184||||||||||||0||||||S||84
|||||||000001|03/01/2022|Vente|143000,00||||B010|CHAMP COCHET|1480|SAVIGNEUX|01|398||

On s'aperçoit que le délimiteur n'est ni un comma (virgure ","), ni une tab (tabulation, "\t").


Ils ont choisi d'utiliser le pipe "|" comme délimiteur

Je ne sais pas comment faire ce pipe sur Mac. Sur ma version de Windows, c'est "AltGR + 6".


Mais ça n'est pas important, je l'ai déjà setup dans la commande de lecture du fichier pour vous:

In [30]:
import pandas as pd
df = pd.read_csv("data/valeursfoncieres-2022.txt", sep="|")

/tmp/ipykernel_1753/3262548704.py:2: DtypeWarning: Columns (18,23,24,26,28,29,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/valeursfoncieres-2022.txt", sep="|")


In [3]:
df.columns

Index(['Identifiant de document', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')

In [4]:
# Tout un tas de colonnes sont nulles, on ne les garde pas
df.isnull().sum()

Identifiant de document       3803885
Reference document            3803885
1 Articles CGI                3803885
2 Articles CGI                3803885
3 Articles CGI                3803885
4 Articles CGI                3803885
5 Articles CGI                3803885
No disposition                      0
Date mutation                       0
Nature mutation                     0
Valeur fonciere                 24721
No voie                       1321227
B/T/Q                         3627918
Type de voie                  1406840
Code voie                       39043
Voie                            39153
Code postal                     39173
Commune                             0
Code departement                    0
Code commune                        0
Prefixe de section            3645046
Section                           155
No plan                             0
No Volume                     3794976
1er lot                       2534572
Surface Carrez du 1er lot     3436147
2eme lot    

In [5]:
# J'ai aussi enlevé d'autres colonnes qui ne nous servent à rien, comme la Nature de la mutation
cols_of_interest = [
    "Date mutation",
    "Valeur fonciere",
    "No voie",
    "Type de voie",
    "Code voie",
    "Voie",
    "Code postal",
    "Commune",
    "Code departement",
    "Code commune",
    "Surface Carrez du 1er lot",
    "Nombre pieces principales",
    "Surface terrain",
]


Puis je garde les données qui concernent les appartements vendus dans le Nord (59) ou dans le Pas de Calais (62)

In [6]:
mask_nord = df["Code departement"] == 59
mask_pdc = df["Code departement"] == 62
mask_appt = df["Type local"] == "Appartement"

appt_nord = df[(mask_nord | mask_pdc) & mask_appt][cols_of_interest]

# 420 apparts pour lesquels on n'a pas le montant de la vente
print(
    appt_nord["Valeur fonciere"]
    .isnull().sum()
)
#drop
appt_nord = appt_nord[ ~appt_nord["Valeur fonciere"].isnull() ]
appt_nord

420


,Date mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Surface Carrez du 1er lot,Nombre pieces principales,Surface terrain
2128057,04/01/2022,"208000,00",64.0,RUE,0010,DE L ABBE BONPAIN,59700.0,MARCQ EN BAROEUL,59,378,"57,69",3.0,NaN
2128060,04/01/2022,"113000,00",1.0,RUE,0260,GUY MOCQUET,59113.0,SECLIN,59,560,"45,72",3.0,NaN
2128064,03/01/2022,"185000,00",22.0,RUE,8749,DE TREVISE,59000.0,LILLE,59,350,"62,83",3.0,NaN
2128065,04/01/2022,"120350,00",132.0,RUE,7554,ROGER SALENGRO,59260.0,LILLE,59,350,"43,64",2.0,NaN
2128072,04/01/2022,"149000,00",7.0,ALL,3430,DES COLOMBIERS,59650.0,VILLENEUVE-D'ASCQ,59,9,"59,02",2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2367475,18/02/2022,"400000,00",8.0,RUE,1540,STE ALDEGONDE,62500.0,SAINT OMER,62,765,NaN,2.0,204.0
2367476,18/02/2022,"400000,00",8.0,RUE,1540,STE ALDEGONDE,62500.0,SAINT OMER,62,765,NaN,2.0,204.0
2367477,18/02/2022,"400000,00",8.0,RUE,1540,STE ALDEGONDE,62500.0,SAINT OMER,62,765,NaN,2.0,204.0
2367478,18/02/2022,"400000,00",8.0,RUE,1540,STE ALDEGONDE,62500.0,SAINT OMER,62,765,NaN,2.0,204.0


Les valeurs foncières sont des nombres avec les virgules au format excel.

Python ne va pas aimer ça, on va les remplacer

In [7]:
appt_nord["Valeur fonciere"] = appt_nord["Valeur fonciere"].apply(lambda x: int(x.split(",")[0]))

## Clean Pandas: All in one

In [36]:
cols_of_interest = [
    "Date mutation",
    "valeur_fonciere",
    "No voie",
    "Type de voie",
    "Code voie",
    "Voie",
    "Code postal",
    "Commune",
    "Code departement",
    "Code commune",
    "Surface Carrez du 1er lot",
    "Nombre pieces principales",
    "Surface terrain",
]

def convert_to_float(valeur_fonciere):
    return float(valeur_fonciere.replace(",", "."))


appt_nord = (
    df.query("`Code departement` in [59, 62] & `Type local` == 'Appartement'")
    #.query("`Valeur fonciere` == `Valeur fonciere`")
    .query("`Valeur fonciere` .notnull()", engine="python")
    .rename({"Valeur fonciere": "valeur_fonciere"}, axis=1)
    .assign(valeur_fonciere=lambda df_: df_.valeur_fonciere.apply(convert_to_float))
    #.drop_duplicates()
)[cols_of_interest]

appt_nord.shape

(18298, 13)

La ligne commentée fait la même chose que la ligne suivante.

On enlève les valeurs nulles.

La raison pour laquelle ça marche, c'est parce qu'un Nan n'est pas égal à lui même:

In [34]:
import numpy as np
np.NaN == np.NaN

False

In [35]:
appt_nord.to_csv("data/appartements_nord_pdc.csv", index=False)

# Performance

In [12]:
%%timeit
(
    df.query("`Code departement` in [59, 62] & `Type local` == 'Appartement'")
)

101 ms ± 2.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit
(
    df.query("`Code departement` in [59, 62]")
    .query("`Type local` == 'Appartement'")
)

74.3 ms ± 858 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%%timeit
(
    df.query("`Type local` == 'Appartement'")
    .query("`Code departement` in [59, 62]")
)

196 ms ± 3.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
import duckdb

Par défaut, By default, DuckDB reads the first 100 lines of a dataframe to determine the data type for Pandas "object" columns. If those 100 lines are null, it might guess the wrong type.

Override this behavior with:


In [16]:
duckdb.default_connection.execute("SET GLOBAL pandas_analyze_sample=100000")

Source: https://stackoverflow.com/questions/75352219/fix-unimplemented-casting-error-in-duckdb-insert

In [17]:
%%timeit
query = """
SELECT * FROM df
WHERE "Code departement" in (59, 62) AND "Type local" = 'Appartement'
"""

duckdb.sql(query)


321 ms ± 4.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
query = """
SELECT * FROM df
WHERE "Type local" = 'Appartement' AND "Code departement" in (59, 62)
"""

duckdb.sql(query)


317 ms ± 2.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


DuckDB irait sans doute beaucoup plus vite si la colonne était entièrement composée de INT. Mais les départements de la CORSE (2A et 2B) empêchent de caster toute la colonne en INT